In [1]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.pooling import GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import itertools
from keras.callbacks import ModelCheckpoint
from scipy import stats

Using TensorFlow backend.
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
from sklearn.model_selection import train_test_split

In [ ]:
X = np.load('data_nosplit/X.npy')
y_onehot = np.load('data_nosplit/y.npy')

In [11]:
def model_CNN_LSTM():
    model = Sequential([
            TimeDistributed(Conv1D(16,3, activation='relu', padding = "same"),input_shape=X.shape[1:]),
            TimeDistributed(BatchNormalization()),
            TimeDistributed(Dropout(0.5)),
            BatchNormalization(),
            TimeDistributed(Flatten()),
            LSTM(20,unit_forget_bias = 0.5, return_sequences = True),
            TimeDistributed(Dense(6,activation='softmax'))])
    adam = Adam(lr=0.0001) 
    model.compile(loss='categorical_crossentropy',
              optimizer= adam, #variants
              metrics=['accuracy'])
    
    return model

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, shuffle = True, test_size = 0.4)
X_test2, X_val, y_test2, y_val = train_test_split(X_test, y_test, shuffle = True, test_size = 0.5)

In [ ]:
model = model_CNN_LSTM()
model_history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val)) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Train on 6605 samples, validate on 2202 samples
Epoch 1/100
6605/6605 [==============================] - 19s 3ms/step - loss: 1.4275 - acc: 0.5308 - val_loss: 0.8700 - val_acc: 0.8648
Epoch 2/100
6605/6605 [==============================] - 17s 3ms/step - loss: 0.6992 - acc: 0.8549 - val_loss: 0.4022 - val_acc: 0.9301
Epoch 3/100
6605/6605 [==============================] - 15s 2ms/step - loss: 0.3957 - acc: 0.9134 - val_loss: 0.2597 - val_acc: 0.9422
Epoch 4/100
6605/6605 [==============================] - 16s 2ms/step - loss: 0.2947 - acc: 0.9296 - val_loss: 0.2063 - val_acc: 0.9538
Epoch 5/100
6605/6605 [====

In [ ]:
for i in range(10):
    
    print(str(i+1) + 'th iteration of for loop')
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, shuffle = True, test_size = 0.4)
    X_test2, X_val, y_test2, y_val = train_test_split(X_test, y_test, shuffle = True, test_size = 0.5)
    model = model_CNN_LSTM()
    
    filepath= 'data_nosplit\Weights'+str(i+1)+'\\'+ "val1" + "-{epoch:02d}-{val_acc:.4f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model_history = model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=callbacks_list, validation_data=(X_val, y_val))
    
    np.save('data_nosplit\Weights'+str(i+1)+'\X_train', X_train)
    np.save('data_nosplit\Weights'+str(i+1)+'\y_train', y_train)
    np.save('data_nosplit\Weights'+str(i+1)+'\X_test', X_test2)
    np.save('data_nosplit\Weights'+str(i+1)+'\y_test', y_test2)
    np.save('data_nosplit\Weights'+str(i+1)+'\X_val', X_val)
    np.save('data_nosplit\Weights'+str(i+1)+'\y_val', y_val)
    
    ## plot accuracy
    acc_name = 'data_nosplit/accuracy_plot'+str(i+1)+'.jpg'
    loss_name = 'data_nosplit/loss_plot'+str(i+1)+'.jpg'
    fig = plt.figure()
    plt.plot(model_history.history['acc'])
    plt.plot(model_history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')
    plt.savefig(acc_name)
    plt.close(fig)

    # # Plot loss
    fig = plt.figure()
    plt.plot(model_history.history['loss'])
    plt.plot(model_history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.savefig(loss_name)
    plt.close(fig)
    

In [ ]:
import pandas as pd
results = np.zeros((10,3))
df = pd.DataFrame(results, columns=['best_model_path', 'validation_accuracy', 'test_accuracy'])

In [ ]:
df['best_model_path'][0] = 'data_nosplit/Weights1/val1-98-0.9963.hdf5'
X_test = np.load('data_nosplit/Weights1/X_test.npy')
y_test = np.load('data_nosplit/Weights1/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][0] = 99.63
df['test_accuracy'][0] = f[1]*100

In [ ]:
df['best_model_path'][1] = 'data_nosplit/Weights2/val1-99-0.9963.hdf5'
X_test = np.load('data_nosplit/Weights2/X_test.npy')
y_test = np.load('data_nosplit/Weights2/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][1] = 99.63
df['test_accuracy'][1] = f[1]*100

In [ ]:
df['best_model_path'][2] = 'data_nosplit/Weights3/val1-90-0.9960.hdf5'
X_test = np.load('data_nosplit/Weights3/X_test.npy')
y_test = np.load('data_nosplit/Weights3/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][2] = 99.60
df['test_accuracy'][2] = f[1]*100

In [ ]:
df['best_model_path'][3] = 'data_nosplit/Weights4/val1-97-0.9971.hdf5'
X_test = np.load('data_nosplit/Weights4/X_test.npy')
y_test = np.load('data_nosplit/Weights4/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][3] = 99.71
df['test_accuracy'][3] = f[1]*100

In [ ]:
df['best_model_path'][4] = 'data_nosplit/Weights5/val1-99-0.9976.hdf5'
X_test = np.load('data_nosplit/Weights5/X_test.npy')
y_test = np.load('data_nosplit/Weights5/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][4] = 99.76
df['test_accuracy'][4] = f[1]*100

In [ ]:
df['best_model_path'][5] = 'data_nosplit/Weights6/val1-97-0.9980.hdf5'
X_test = np.load('data_nosplit/Weights6/X_test.npy')
y_test = np.load('data_nosplit/Weights6/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][5] = 99.80
df['test_accuracy'][5] = f[1]*100

In [ ]:
df['best_model_path'][6] = 'data_nosplit/Weights7/val1-89-0.9958.hdf5'
X_test = np.load('data_nosplit/Weights7/X_test.npy')
y_test = np.load('data_nosplit/Weights7/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][6] = 99.58
df['test_accuracy'][6] = f[1]*100

In [ ]:
df['best_model_path'][7] = 'data_nosplit/Weights8/val1-88-0.9966.hdf5'
X_test = np.load('data_nosplit/Weights8/X_test.npy')
y_test = np.load('data_nosplit/Weights8/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][7] = 99.66
df['test_accuracy'][7] = f[1]*100

In [ ]:
df['best_model_path'][8] = 'data_nosplit/Weights9/val1-82-0.9966.hdf5'
X_test = np.load('data_nosplit/Weights9/X_test.npy')
y_test = np.load('data_nosplit/Weights9/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][8] = 99.66
df['test_accuracy'][8] = f[1]*100

In [ ]:
df['best_model_path'][9] = 'data_nosplit/Weights10/val1-99-0.9966.hdf5'
X_test = np.load('data_nosplit/Weights10/X_test.npy')
y_test = np.load('data_nosplit/Weights10/y_test.npy')
f = model.evaluate(X_test, y_test)
df['validation_accuracy'][9] = 99.66
df['test_accuracy'][9] = f[1]*100